In [94]:
import json
from tqdm import tqdm
import itertools
import pdb
import glob
import os
from PIL import Image
from natsort import natsorted

# Utilities

In [97]:
def divid_train_val_image(image_dir, img_ext, val_ratio):
    files = glob.glob(os.path.join(image_dir, "*" + img_ext))
    
    files = natsorted(files)
    val_files = []
    train_files = []
    val_ratio_rpl = 1/val_ratio
    sum_ration = 0
    for idx, file in enumerate(files):
        sum_ration += val_ratio
        if sum_ration > 1:
            sum_ration = 0
            val_files.append(file)
        else:
            train_files.append(file)
            
    return train_files, val_files
    
    


In [96]:
image_dir = r"L:\Datasets\ocean_city\processed_json"
img_ext = ".json"
val_ratio = 0.2

train_files, val_files = divid_train_val_image(image_dir, img_ext, val_ratio)
print(len(train_files), len(val_files))

138 27


In [98]:
files = glob.glob(os.path.join(image_dir, "*" + img_ext))

In [99]:
files = glob.glob(os.path.join(r"L:\Datasets\ocean_city\processed_json\*.json"))
files = natsorted(files)

# Create image json

In [46]:
def create_image_json(files):
#     with open('gqa_image_metadata.json', 'w') as outfile:  
#     json.dump(gqa_image_data, outfile)
    results = []
    for file in files:
        file = file.replace(".json", ".jpg")
        img = Image.open(file)
        
        img_name = os.path.basename(file).replace(".jpg", "")
        
        image_num = img_name.split("_")[0]
        image_num = int(image_num)
        
        img_h, img_w = img.size
        
        # print(img_w, img_h)
        data_dict = {"image_id": img_name,
                     "image_num": image_num,
                     "width": img_w,
                     "height": img_h,
                    "url": None, 
                    "flickr_id": None,
                    "coco_id": None,
                    "anti_prop": 0.0}
        results.append(data_dict)
        
    return results
        
#     print(results)


# for image_set in (train_files, val_files):
jdata = create_image_json(train_files)
with open(r'L:\Datasets\ocean_city\image_train.json', 'w') as outfile:  
    json.dump(jdata, outfile, indent=2)
    
jdata = create_image_json(val_files)
with open(r'L:\Datasets\ocean_city\image_val.json', 'w') as outfile:  
    json.dump(jdata, outfile, indent=2)    

# Create object json

In [72]:
def create_obj_dict(json_list, obj_list):
    
    for file in json_list:
        f = open(file, 'r')
        jdata = json.load(f)
                 
        shapes = jdata['shapes']
        image_id = os.path.basename(file).replace(".json", "")
        image_num = image_id.split("_")[0]
        image_url = ""
        objects = []
         
                 
        tmp_dict = {'image_id' : image_id, 
                    "image_num": image_num,
                    'objects' : [], 
                    'image_url' : image_url}
                                    
        for shape in shapes:
            points = shape['points']
            points = np.array(points)
                 
            x = int(min(points[:, 0]))
            y = int(min(points[:, 1]) )                 
            w = int(max(points[:, 0]) - min(points[:, 0]))
            h = int(max(points[:, 1]) - min(points[:, 1]) )
            name = shape["label"]
            object_id = shape["object_id"]
            
            object_dict = {'names': shape['label'],
                           'object_id': object_id,
                           'merged_object_ids': [],
                           'synsets': [shape['label']],                         
                           'x': x,                           
                           'y': y,
                           'w': w,
                           'h': h}
            objects.append(object_dict)

            tmp_dict['objects'] = objects     
            
            obj_list.append(tmp_dict)
            
        
    return obj_list

obj_list = []

create_obj_dict(train_files, obj_list)

with open(r'L:\Datasets\ocean_city\objects_train.json', 'w') as outfile:  
    json.dump(obj_list, outfile, indent=2)

obj_list = []
create_obj_dict(val_files, obj_list)
with open(r'L:\Datasets\ocean_city\objects_val.json', 'w') as outfile:  
    json.dump(obj_list, outfile, indent=2)  

# create relationships

In [100]:
def create_rel_dict(json_list, rel_list):
    
    for file in json_list:
        f = open(file, 'r')
        print("Processing: ", file)
        jdata = json.load(f)
                 
        shapes = jdata['shapes']
        image_id = os.path.basename(file).replace(".json", "")
        image_num = image_id.split("_")[0]
        image_url = ""
        objects = []
        
        obj_list = []
         
        object_ids = []
        
        tmp_dict = {'image_id' : image_id, 
                    "image_num": image_num,
                    'objects' : [], 
                    'image_url' : image_url}
            
                                    
        for shape in shapes:
            points = shape['points']
            points = np.array(points)
                 
            x = int(min(points[:, 0]))
            y = int(min(points[:, 1]) )                 
            w = int(max(points[:, 0]) - min(points[:, 0]))
            h = int(max(points[:, 1]) - min(points[:, 1]) )
            name = shape["label"]
            object_id = shape["object_id"]
            
            object_ids.append(object_id)
            
            object_dict = {'names': shape['label'],
                           'object_id': object_id,
                           'merged_object_ids': [],
                           'synsets': [shape['label']],                         
                           'x': x,                           
                           'y': y,
                           'w': w,
                           'h': h}
            objects.append(object_dict)

            tmp_dict['objects'] = objects     
            
            obj_list.append(tmp_dict)
            
        relationships = jdata["relationships"]
        
#         print(object_ids)
        
        rel_dict = {}
        for rel in relationships:
            
            rel_dict["relationship_id"] = rel["relationship_id"]
#             subject_name = rel["subject"]["name"] 
            subject_id   = rel["subject"]["object_id"]
            subject_id = int(subject_id)
#             print("subject_id:", subject_id)
#             oject_name   = rel["object"]["name"] 
            object_id    = rel["object"]["object_id"]
            object_id = int(object_id)
            predicate    = rel["predicate"]
            
            subject_idx = object_ids.index(subject_id)
            object_idx = object_ids.index(object_id)
            
            subject_dict = obj_list[subject_idx]
            object_dict = obj_list[object_idx]
            
            rel_dict["synsets"] = []
            rel_dict["subject"] = subject_dict
            rel_dict["object"]  = object_dict
            
            rel_list.append(rel_dict)
            
        
    return rel_list

rel_list = []

create_rel_dict(train_files, rel_list)

with open(r'L:\Datasets\ocean_city\relationships_train.json', 'w') as outfile:  
    json.dump(rel_list, outfile, indent=2)

rel_list = []
create_rel_dict(val_files, rel_list)
with open(r'L:\Datasets\ocean_city\relationships_val.json', 'w') as outfile:  
    json.dump(rel_list, outfile, indent=2)  

Processing:  L:\Datasets\ocean_city\processed_json\1_aLH8om-ax5boXJ-hXhFvaQ_123.73.json
Processing:  L:\Datasets\ocean_city\processed_json\2_N2TakAa4EvhSCDIb44lEBw_83.71.json
Processing:  L:\Datasets\ocean_city\processed_json\3_n4aTTgHhJ2AkPbSg0X3MfA_11.99.json
Processing:  L:\Datasets\ocean_city\processed_json\4_7pBBmj5k9GAh1o-n4ctSpQ_56.94.json
Processing:  L:\Datasets\ocean_city\processed_json\5_0_Z25VniDxurxsixrPV1LA_33.93.json
Processing:  L:\Datasets\ocean_city\processed_json\8_M3I2qNQeBB2MFg3IHuCbig_104.0.json
Processing:  L:\Datasets\ocean_city\processed_json\9_as2lf1LPPhGI1D5DEQtDyA_55.07.json
Processing:  L:\Datasets\ocean_city\processed_json\10_piJ3-HBctCPJ9qejZZesEQ_121.73.json
Processing:  L:\Datasets\ocean_city\processed_json\11_eD6yAl7EAam1xa8Z6_5Amw_31.49.json
Processing:  L:\Datasets\ocean_city\processed_json\13_xixwgkh4GQ9uyj7DHwqcgw_54.73.json
Processing:  L:\Datasets\ocean_city\processed_json\15_IU40Fjvi2newL6AnPrtlvQ_128.5.json
Processing:  L:\Datasets\ocean_city\p

Processing:  L:\Datasets\ocean_city\processed_json\165_uMgYUtFt1-s-Y04jeoHJ7A_142.33.json
Processing:  L:\Datasets\ocean_city\processed_json\175_7i9_4f8BENdEjVUgxY3KAQ_285.41.json
Processing:  L:\Datasets\ocean_city\processed_json\176_6POgsLbygPA3LoniFTL9mA_319.71.json
Processing:  L:\Datasets\ocean_city\processed_json\177_RM7iP31lL7HfN5yQk3EwjQ_314.68.json
Processing:  L:\Datasets\ocean_city\processed_json\178_33wz2rVhX68hyfkPCRW6lQ_137.74.json
Processing:  L:\Datasets\ocean_city\processed_json\180_1JZly-kT66kQLUKOa8WO4w_129.61.json
Processing:  L:\Datasets\ocean_city\processed_json\185_svuylq4Uz0V_HJFYDQ7hyg_313.65.json
Processing:  L:\Datasets\ocean_city\processed_json\186_oxBJpfvEW1X8aiy4CA6H_w_317.07.json
Processing:  L:\Datasets\ocean_city\processed_json\187_3UmBXeVJE9lO6Te8iIN3Uw_297.0.json
Processing:  L:\Datasets\ocean_city\processed_json\188_3UmBXeVJE9lO6Te8iIN3Uw_333.43.json
Processing:  L:\Datasets\ocean_city\processed_json\189_GoF92nFLpEhiGtKGjsDSLw_304.79.json
Processing:

# look up json

In [82]:
json_file = r'L:\Datasets\Visual_Genome\relationships.json'

f = open(json_file, 'r')

jdata = json.load(f)



In [83]:
len(jdata)

108077

In [84]:
jdata[0]

{'relationships': [{'predicate': 'ON',
   'object': {'h': 290,
    'object_id': 1058534,
    'merged_object_ids': [5046],
    'synsets': ['sidewalk.n.01'],
    'w': 722,
    'y': 308,
    'x': 78,
    'names': ['sidewalk']},
   'relationship_id': 15927,
   'synsets': ['along.r.01'],
   'subject': {'name': 'shade',
    'h': 192,
    'synsets': ['shade.n.01'],
    'object_id': 5045,
    'w': 274,
    'y': 338,
    'x': 119}},
  {'predicate': 'wears',
   'object': {'h': 28,
    'object_id': 1058525,
    'merged_object_ids': [5048],
    'synsets': ['shoe.n.01'],
    'w': 48,
    'y': 485,
    'x': 388,
    'names': ['shoes']},
   'relationship_id': 15928,
   'synsets': ['wear.v.01'],
   'subject': {'name': 'man',
    'h': 262,
    'synsets': ['man.n.01'],
    'object_id': 1058529,
    'w': 60,
    'y': 249,
    'x': 238}},
  {'predicate': 'has',
   'object': {'name': 'headlight',
    'h': 15,
    'synsets': ['headlight.n.01'],
    'object_id': 5050,
    'w': 23,
    'y': 366,
    'x': 514}

In [12]:
def create_gqa_image_data(gqa_image_data, scene_graphs, forb_list):

    for img in scene_graphs.keys():
        if img not in forb_list:
            gqa_image_data.append({'width': scene_graphs[img]['width'],
                                   'url' : None,
                                   'height': scene_graphs[img]['height'],
                                   'image_id': int(img),
                                   'coco_id' : None,
                                   'flickr_id' : None,
                                   'anti_prop' : 0.0})
    return gqa_image_data

In [3]:
gqa_image_data = []

for name in ('val_sceneGraphs', 'train_sceneGraphs'):
    with open('//local-scratch/data/GQA/SceneGraphs/' + name + '.json') as json_file:
        obj_data = json.load(json_file)    
    
    gqa_image_data = create_gqa_image_data(gqa_image_data, obj_data, [])

with open('gqa_image_metadata.json', 'w') as outfile:  
    json.dump(gqa_image_data, outfile)
    

In [4]:


def create_obj_dict(img_list, obj_data, forb_list):
    
    for img in obj_data.keys():
        if img not in forb_list:
            tmp_dict = {'image_id' : int(img), 'objects' : [], 'image_url' : ''}
            tmp_list = []
            for obj in obj_data[img]['objects']:
                tmp_list.append({'synsets': [obj_data[img]['objects'][obj]['name']],
                                 'h': obj_data[img]['objects'][obj]['h'],
                                 'object_id': int(obj),
                                 'merged_object_ids': [],
                                 'names': [obj_data[img]['objects'][obj]['name']],
                                 'w': obj_data[img]['objects'][obj]['w'],
                                 'y': obj_data[img]['objects'][obj]['y'],
                                 'x': obj_data[img]['objects'][obj]['x']})

            tmp_dict['objects'] = tmp_list
            img_list.append(tmp_dict)
        
    return img_list

In [5]:
def create_rel_dict(cntr, rel_list, obj_data, forb_list):

    for img in obj_data.keys():
        if img not in forb_list:
            tmp_list = []
            for obj in obj_data[img]['objects']:
                for rel in obj_data[img]['objects'][obj]['relations']:
                    cntr += 1
                    subj = rel['object']
                    tmp_list.append({'predicate': rel['name'],
                                     'object': {'h': obj_data[img]['objects'][obj]['h'],
                                     'object_id': int(obj),
                                     'merged_object_ids': [],
                                     'synsets': [obj_data[img]['objects'][obj]['name']],
                                     'w': obj_data[img]['objects'][obj]['w'],
                                     'y': obj_data[img]['objects'][obj]['y'],
                                     'x': obj_data[img]['objects'][obj]['x'],
                                     'names': [obj_data[img]['objects'][obj]['name']]},
                                    'relationship_id': cntr,
                                    'synsets': [rel['name']],
                                    'subject': {'name': obj_data[img]['objects'][subj]['name'],
                                     'h': obj_data[img]['objects'][subj]['h'],
                                     'synsets': [obj_data[img]['objects'][subj]['name']],
                                     'object_id': int(subj),
                                     'w': obj_data[img]['objects'][subj]['w'],
                                     'y': obj_data[img]['objects'][subj]['y'],
                                     'x': obj_data[img]['objects'][subj]['x']}})
            rel_list.append({'relationships' : tmp_list, 'image_id' : int(img)})
    
    return cntr, rel_list

In [6]:
def create_attr_dict(attr_list, obj_data, forb_list):
    
    for img in obj_data.keys():
        if img not in forb_list:
            tmp_dict = {'image_id' : int(img), 'attributes' : []}
            tmp_list = []
            for obj in obj_data[img]['objects']:
                if obj_data[img]['objects'][obj]['attributes'] != []:
                    tmp_list.append({'synsets': [obj_data[img]['objects'][obj]['name']],
                                     'h': obj_data[img]['objects'][obj]['h'],
                                     'object_id': int(obj),
                                     'attributes' : obj_data[img]['objects'][obj]['attributes'],
                                     'names': [obj_data[img]['objects'][obj]['name']],
                                     'w': obj_data[img]['objects'][obj]['w'],
                                     'y': obj_data[img]['objects'][obj]['y'],
                                     'x': obj_data[img]['objects'][obj]['x']})
                else:
                    tmp_list.append({'synsets': [obj_data[img]['objects'][obj]['name']],
                                     'h': obj_data[img]['objects'][obj]['h'],
                                     'object_id': int(obj),
                                     'names': [obj_data[img]['objects'][obj]['name']],
                                     'w': obj_data[img]['objects'][obj]['w'],
                                     'y': obj_data[img]['objects'][obj]['y'],
                                     'x': obj_data[img]['objects'][obj]['x']})

            tmp_dict['attributes'] = tmp_list
            attr_list.append(tmp_dict)
        
    return attr_list

In [7]:
def save_to_file(text, name):

    with open(name, mode='wt', encoding='utf-8') as myfile:
        myfile.write('\n'.join(text))
        myfile.write('\n')

In [9]:
cntr = 0
img_list = []
rel_list = []
attr_list = []
gqa_image_data = []
forb_list = open("non_ex_haha.txt", "r").read().split('\n')

for name in ('val_sceneGraphs', 'train_sceneGraphs'):
    with open('//local-scratch/data/GQA/SceneGraphs/' + name + '.json') as json_file:
        obj_data = json.load(json_file)
        
    img_list = create_obj_dict(img_list, obj_data, forb_list)
    cntr, rel_list = create_rel_dict(cntr, rel_list, obj_data, forb_list)
    gqa_image_data = create_gqa_image_data(gqa_image_data, obj_data, forb_list)
    attr_list = create_attr_dict(attr_list, obj_data, forb_list)

In [10]:
relsMap = json.load(open('/home/lkochiev/Documents/SFU/NSM/vgLists/relInfo.json'))
save_to_file(list(relsMap.keys()), 'gqa_relationship_list.txt')

In [11]:
attrInfo = json.load(open('/home/lkochiev/Documents/SFU/NSM/vgLists/attrMap.json'))
merged = list(itertools.chain(*list(attrInfo.values())))
save_to_file(merged, 'gqa_attribute_list.txt')

In [12]:
catMap = json.load(open('/home/lkochiev/Documents/SFU/NSM/vgLists/hObjInfo.json'))
save_to_file(list(catMap.keys()), 'gqa_object_list.txt')

In [13]:
with open('gqa_objects.json', 'w') as outfile:
    json.dump(img_list, outfile)
    
with open('gqa_relationships.json', 'w') as outfile:
    json.dump(rel_list, outfile)
    
with open('gqa_image_metadata.json', 'w') as outfile:  
    json.dump(gqa_image_data, outfile)
    
with open('gqa_attributes.json', 'w') as outfile:
    json.dump(attr_list, outfile)

In [38]:
obj_ent = []
att_ent = []
rel_ent = []

for name in ('val_sceneGraphs', 'train_sceneGraphs'):
    with open('//local-scratch/data/GQA/SceneGraphs/' + name + '.json') as json_file:
        obj_data = json.load(json_file)
    
    for key in tqdm(obj_data.keys()):
        for obj in obj_data[key]['objects']:
            
            if obj_data[key]['objects'][obj]['name'] in obj_ent:
                continue
            else: 
                obj_ent.append(obj_data[key]['objects'][obj]['name'])
            
            for rel in obj_data[key]['objects'][obj]['relations']:
                if rel['name'] == 'licking':
                    print('licking')
                if rel['name'] in rel_ent:
                    continue
                else: 
                    rel_ent.append(rel['name'])
                    
            for att in obj_data[key]['objects'][obj]['attributes']:
                if att in att_ent:
                    continue
                else: 
                    att_ent.append(att)

100%|██████████| 74942/74942 [00:03<00:00, 24782.53it/s]


In [40]:
len(rel_ent), len(att_ent)

(65, 182)